In [1]:
from playwright.async_api import async_playwright

p = await async_playwright().start()

In [2]:
browser = await p.chromium.launch(headless=False)

In [ ]:
page = await browser.new_page(viewport={'width': 1366, 'height': 768})

In [4]:
await page.goto("https://globo.com")

<Response url='https://www.globo.com/' request=<Request url='https://www.globo.com/' method='GET'>>

In [5]:
s = await page.screenshot()
import base64

img = base64.b64encode(s)
img.decode()

'...'

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, ImageContentBlock, ToolMessage
import os

# Configurar modelo via HuggingFace Router
model = ChatOpenAI(
    model="moonshotai/Kimi-K2.5:novita", 
    api_key=os.environ["HF_TOKEN"], 
    base_url="https://router.huggingface.co/v1"
)

# FORMATO CORRETO - use 'content' não 'content_blocks'
message = ToolMessage(
    content=[
        {
            "type": "text",
            "text": "Descreva o que você vê nesta imagem"
        },
        ImageContentBlock(type='image', mime_type='image/png', base64=img.decode())
        # {
        #     "type": "image_url",
        #     "image_url": {
        #         "url": f"data:image/png;base64,{img.decode()}"
        #     }
        # }
    ]
)

# Invocar o modelo
response = model.invoke([message])
print(response.content)


KeyError: 'tool_call_id'

In [82]:
import base64
from langchain_core.messages import HumanMessage

# s é bytes do PNG (resultado do page.screenshot())
# Converte bytes para base64 string

base64_image = base64.b64encode(s).decode('utf-8')

# Cria a mensagem com a imagem
human_message = HumanMessage(
    content=[
        
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_image}"
            }
        },
        {"type": "text", "text": "Descreva o que você vê nesta imagem"}
    ]
)

In [83]:
base64_image

'...'

In [84]:
model.invoke([human_message])

AIMessage(content='', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 512, 'prompt_tokens': 1221, 'total_tokens': 1733}, 'model_name': 'moonshotai/Kimi-K2.5', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None}, id='lc_run--019c11b6-a0a5-7601-8902-86c8e93d84d0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1221, 'output_tokens': 512, 'total_tokens': 1733})

In [81]:
await browser.close()